#Análisis de Datos Geoespaciales - Centros Poblados (CCPP)

Este notebook está diseñado para **extraer, explorar y visualizar** los datos geoespaciales contenidos en la carpeta `CCPP_0`.

##  **Tipos de Archivos Detectados:**

Los archivos en tu carpeta `CCPP_0` son **Shapefiles**, un formato estándar para datos vectoriales geoespaciales:



## 📊 CARGA Y EXPLORACIÓN DE DATOS SHAPEFILE

Ahora que tenemos el entorno configurado, vamos a cargar y explorar los datos de Centros Poblados.

## 📋 EXPLORACIÓN DE ATRIBUTOS Y COORDENADAS

Los archivos Shapefile contienen **AMBOS**: datos tabulares (columnas) Y geometrías (coordenadas). Vamos a explorar qué información contienen usando métodos alternativos.

In [1]:
# 🔍 EXPLORAR ATRIBUTOS USANDO ARCHIVO .DBF
print("🔍 EXPLORANDO ATRIBUTOS Y COORDENADAS DEL SHAPEFILE")
print("=" * 60)


# Vamos a leer el archivo .dbf que contiene los atributos
DBF_PATH = '../CCPP_0/CCPP_IGN100K.dbf'

print("📋 MÉTODO 1: Explorando archivo .DBF (datos tabulares)")
print("-" * 50)

try:
    # Intentar con pandas y diferentes encoding
    encodings_to_try = ['utf-8', 'latin1', 'cp1252', 'iso-8859-1']
    
    for encoding in encodings_to_try:
        try:
            # Usando pandas para leer DBF como archivo binario
            print(f"🔄 Intentando encoding: {encoding}")
            
            # Método alternativo: usar struct para leer DBF
            import struct
            
            with open(DBF_PATH, 'rb') as f:
                # Leer header del DBF
                header = f.read(32)
                if len(header) >= 32:
                    # Extraer información del header
                    version = header[0]
                    year = header[1] + 1900 if header[1] > 0 else 2000 + header[1]
                    month = header[2]
                    day = header[3]
                    num_records = struct.unpack('<L', header[4:8])[0]
                    header_length = struct.unpack('<H', header[8:10])[0]
                    record_length = struct.unpack('<H', header[10:12])[0]
                    
                    print(f"✅ DBF Header leído exitosamente!")
                    print(f"   📊 Número de registros: {num_records:,}")
                    print(f"   📏 Longitud de registro: {record_length} bytes")
                    print(f"   📅 Fecha de modificación: {day}/{month}/{year}")
                    
                    # Calcular número de campos
                    num_fields = (header_length - 33) // 32
                    print(f"   📋 Número de campos/columnas: {num_fields}")
                    
                    # Leer descriptores de campos
                    print(f"\n📝 COLUMNAS DETECTADAS:")
                    fields = []
                    for i in range(num_fields):
                        field_desc = f.read(32)
                        if len(field_desc) >= 32:
                            field_name = field_desc[:11].decode('utf-8', errors='ignore').rstrip('\x00')
                            field_type = chr(field_desc[11])
                            field_length = field_desc[16]
                            field_decimal = field_desc[17]
                            
                            fields.append({
                                'name': field_name,
                                'type': field_type,
                                'length': field_length,
                                'decimal': field_decimal
                            })
                            
                            print(f"   {i+1:2d}. {field_name:<12} | Tipo: {field_type} | Longitud: {field_length}")
                    
                    break  # Si llegamos aquí, fue exitoso
            
        except Exception as e:
            print(f"   ❌ Error con {encoding}: {str(e)[:50]}...")
            continue
    
    print(f"\n🎯 RESUMEN DE CONTENIDO:")
    print(f"   • El shapefile contiene {num_records:,} centros poblados")
    print(f"   • Cada centro poblado tiene {num_fields} atributos/columnas")
    print(f"   • Además de coordenadas geográficas (geometría)")
    
except Exception as e:
    print(f"❌ Error general: {e}")
    print("🔧 Intentando método alternativo...")

print(f"\n💡 EXPLICACIÓN:")
print(f"   🗂️  Los archivos Shapefile son HÍBRIDOS:")
print(f"       📊 Datos tabulares (como CSV) = atributos de cada lugar")
print(f"       🗺️  Datos geoespaciales = coordenadas y geometrías") 
print(f"   📍 Cada fila representa un centro poblado CON:")
print(f"       📝 Atributos (nombre, código, población, etc.)")
print(f"       🌍 Coordenadas geográficas (latitud, longitud)")

🔍 EXPLORANDO ATRIBUTOS Y COORDENADAS DEL SHAPEFILE
📋 MÉTODO 1: Explorando archivo .DBF (datos tabulares)
--------------------------------------------------
🔄 Intentando encoding: utf-8
✅ DBF Header leído exitosamente!
   📊 Número de registros: 136,587
   📏 Longitud de registro: 418 bytes
   📅 Fecha de modificación: 24/9/2018
   📋 Número de campos/columnas: 13

📝 COLUMNAS DETECTADAS:
    1. OBJECTID     | Tipo: N | Longitud: 10
    2. NOM_POBLAD   | Tipo: C | Longitud: 60
    3. FUENTE       | Tipo: C | Longitud: 5
    4. CÓDIGO       | Tipo: C | Longitud: 10
    5. CAT_POBLAD   | Tipo: C | Longitud: 25
    6. DIST         | Tipo: C | Longitud: 60
    7. PROV         | Tipo: C | Longitud: 50
    8. DEP          | Tipo: C | Longitud: 50
    9. CÓD_INT      | Tipo: C | Longitud: 4
   10. CATEGORIA    | Tipo: C | Longitud: 25
   11. X            | Tipo: N | Longitud: 19
   12. Y            | Tipo: N | Longitud: 19
   13. N_BUSQDA     | Tipo: C | Longitud: 80

🎯 RESUMEN DE CONTENIDO:
   • E

In [8]:
# 📊 EXTRAER COORDENADAS Y DATOS ESPECÍFICOS
print("📊 EXTRAYENDO COORDENADAS Y DATOS DE LOS CENTROS POBLADOS")
print("=" * 70)

# Intentar extraer una muestra de datos del DBF
print("🔄 Extrayendo muestra de datos...")

try:
    with open(DBF_PATH, 'rb') as f:
        # Saltar header (32 bytes) + descriptores de campos (13 * 32 bytes) + terminador (1 byte)
        header_size = 32 + 13 * 32 + 1
        f.seek(header_size)
        
        print(f"📋 MUESTRA DE DATOS (primeros 5 registros):")
        print("-" * 70)
        
        for record_num in range(min(5, num_records)):
            # Leer un registro
            record = f.read(record_length)
            if len(record) >= record_length:
                # Procesar el registro (simplificado)
                # El primer byte es el indicador de borrado
                if record[0] != ord('*'):  # No está borrado
                    
                    # Extraer campos básicos (aproximado, puede variar según la estructura exacta)
                    try:
                        # Campo OBJECTID (posición aproximada)
                        object_id = record[1:11].decode('utf-8', errors='ignore').strip()
                        
                        # Campo NOM_POBLAD (posición aproximada)
                        nom_poblad = record[11:71].decode('utf-8', errors='ignore').strip()
                        
                        # Campo X (coordenada - posición aproximada)
                        x_start = 1 + 10 + 60 + 5 + 10 + 25 + 60 + 50 + 50 + 4 + 25  # Suma de longitudes anteriores
                        x_raw = record[x_start:x_start+19].decode('utf-8', errors='ignore').strip()
                        
                        # Campo Y (coordenada - posición aproximada)
                        y_start = x_start + 19
                        y_raw = record[y_start:y_start+19].decode('utf-8', errors='ignore').strip()
                        
                        print(f"   {record_num + 1}. ID: {object_id}")
                        print(f"      📍 Nombre: {nom_poblad[:40]}{'...' if len(nom_poblad) > 40 else ''}")
                        print(f"      🌍 Coordenada X: {x_raw}")
                        print(f"      🌍 Coordenada Y: {y_raw}")
                        print()
                        
                    except Exception as e:
                        print(f"      ⚠️  Error procesando registro {record_num + 1}: {str(e)[:30]}...")
                        
    print("✅ Extracción de muestra completada!")
    
except Exception as e:
    print(f"❌ Error extrayendo datos: {e}")

print(f"\n🗺️ INFORMACIÓN SOBRE LAS COORDENADAS:")
print(f"   📐 Las columnas X e Y contienen coordenadas geográficas")
print(f"   🌍 Estas coordenadas representan la ubicación exacta de cada centro poblado")
print(f"   📊 Hay {num_records:,} centros poblados con sus respectivas coordenadas")

print(f"\n📋 COLUMNAS DISPONIBLES PARA ANÁLISIS:")
print(f"   1. 📍 NOM_POBLAD - Nombre del centro poblado")
print(f"   2. 🏛️  DIST - Distrito")
print(f"   3. 🗺️  PROV - Provincia") 
print(f"   4. 🏴 DEP - Departamento")
print(f"   5. 🏷️  CATEGORIA - Categoría del centro poblado")
print(f"   6. 🌍 X, Y - Coordenadas geográficas (CLAVE PARA TU ANÁLISIS)")

print(f"\n🎯 PARA TU ANÁLISIS DE RIESGO HÍDRICO:")
print(f"   • Puedes usar las coordenadas X, Y para ubicar geográficamente los centros poblados")
print(f"   • Cruzar con tus datos de monitoreo de agua usando proximidad geográfica")
print(f"   • Identificar qué centros poblados están cerca de zonas de alto riesgo")
print(f"   • Analizar por departamento, provincia o distrito")

📊 EXTRAYENDO COORDENADAS Y DATOS DE LOS CENTROS POBLADOS
🔄 Extrayendo muestra de datos...
📋 MUESTRA DE DATOS (primeros 5 registros):
----------------------------------------------------------------------
   1. ID: 1
      📍 Nombre: PANDISHARI
      🌍 Coordenada X: -74.06462000
      🌍 Coordenada Y: -10.37129000

   2. ID: 2
      📍 Nombre: CHICOSA
      🌍 Coordenada X: -74.06153000
      🌍 Coordenada Y: -10.37852000

   3. ID: 3
      📍 Nombre: RAYA
      🌍 Coordenada X: -72.94118000
      🌍 Coordenada Y: -10.33043000

   4. ID: 4
      📍 Nombre: PENSILVANIA
      🌍 Coordenada X: -74.05988000
      🌍 Coordenada Y: -10.40401000

   5. ID: 5
      📍 Nombre: PONTE VEDRA
      🌍 Coordenada X: -74.03788000
      🌍 Coordenada Y: -10.41809000

✅ Extracción de muestra completada!

🗺️ INFORMACIÓN SOBRE LAS COORDENADAS:
   📐 Las columnas X e Y contienen coordenadas geográficas
   🌍 Estas coordenadas representan la ubicación exacta de cada centro poblado
   📊 Hay 136,587 centros poblados con sus 

In [3]:
# 💾 CONVERTIR A FORMATO TABULAR (CSV/DataFrame)
print("💾 CONVIRTIENDO SHAPEFILE A FORMATO TABULAR")
print("=" * 60)

# Definir la ruta del shapefile
SHAPEFILE_PATH = '../CCPP_0/CCPP_IGN100K.shp'

# Instalar y usar pyshp como alternativa más simple
print("🔄 Instalando pyshp para leer shapefiles...")
import subprocess
import sys

try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pyshp"])
    print("✅ pyshp instalado exitosamente")
    
    import shapefile
    import pandas as pd
    
    # Leer el shapefile
    print("🔄 Leyendo shapefile con pyshp...")
    
    with shapefile.Reader(SHAPEFILE_PATH) as shp:
        # Obtener información básica
        print(f"✅ Shapefile abierto exitosamente!")
        print(f"📊 Número de registros: {len(shp):,}")
        print(f"📐 Tipo de geometría: {shp.shapeType}")
        print(f"📏 Límites: {shp.bbox}")
        
        # Obtener nombres de campos
        field_names = [field[0] for field in shp.fields[1:]]  # El primer campo es siempre 'DeletionFlag'
        print(f"📋 Campos: {field_names}")
        
        # Crear DataFrame con una muestra
        print(f"\n🔄 Extrayendo muestra de 1000 registros...")
        
        records_sample = []
        coords_sample = []
        
        # Leer una muestra de registros
        for i, (record, shape) in enumerate(zip(shp.records(), shp.shapes())):
            if i >= 1000:  # Solo procesar 1000 registros como muestra
                break
                
            # Extraer coordenadas del punto
            if shape.points:
                x, y = shape.points[0]  # Primer punto (para puntos, solo hay uno)
                coords_sample.append([x, y])
                
                # Agregar record con coordenadas
                record_dict = dict(zip(field_names, record))
                record_dict['COORD_X'] = x
                record_dict['COORD_Y'] = y
                records_sample.append(record_dict)
        
        # Crear DataFrame
        df_ccpp_sample = pd.DataFrame(records_sample)
        
        print(f"✅ Muestra extraída: {len(df_ccpp_sample):,} registros")
        print(f"📋 Columnas disponibles: {list(df_ccpp_sample.columns)}")
        
        # Mostrar información de la muestra
        print(f"\n📊 INFORMACIÓN DE LA MUESTRA:")
        print(f"   📍 Rango de coordenadas X: {df_ccpp_sample['COORD_X'].min():.5f} a {df_ccpp_sample['COORD_X'].max():.5f}")
        print(f"   📍 Rango de coordenadas Y: {df_ccpp_sample['COORD_Y'].min():.5f} a {df_ccpp_sample['COORD_Y'].max():.5f}")
        
        # Mostrar primeras filas
        print(f"\n👀 PRIMERAS 5 FILAS:")
        display_cols = ['NOM_POBLAD', 'DIST', 'DEP', 'COORD_X', 'COORD_Y']
        print(df_ccpp_sample[display_cols].head())
        
        # Análisis por departamento
        print(f"\n🏴 DISTRIBUCIÓN POR DEPARTAMENTO (Top 10):")
        dep_counts = df_ccpp_sample['DEP'].value_counts().head(10)
        for dep, count in dep_counts.items():
            print(f"   • {dep}: {count:,} centros poblados")
        
        print(f"\n💾 OPCIONES PARA EXPORTAR:")
        print(f"   1. 💿 Guardar como CSV: df_ccpp_sample.to_csv('centros_poblados.csv')")
        print(f"   2. 🔗 Usar para análisis geoespacial con tus datos de agua")
        print(f"   3. 📊 Crear visualizaciones de distribución geográfica")
        
except Exception as e:
    print(f"❌ Error: {e}")
    print("💡 Método alternativo disponible si es necesario")

💾 CONVIRTIENDO SHAPEFILE A FORMATO TABULAR
🔄 Instalando pyshp para leer shapefiles...
✅ pyshp instalado exitosamente
✅ pyshp instalado exitosamente



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 711, in start
    self.io_loop.start()
  Fil

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 711, in start
    self.io_loop.start()
  Fil

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 711, in start
    self.io_loop.start()
  Fil

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/Users/jleandrojm/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 711, in start
    self.io_loop.start()
  Fil

AttributeError: _ARRAY_API not found

🔄 Leyendo shapefile con pyshp...
✅ Shapefile abierto exitosamente!
📊 Número de registros: 136,587
📐 Tipo de geometría: 1
📏 Límites: [-81.31011402699994, -18.344071732999964, -68.65995945699996, -0.030712728999958472]
📋 Campos: ['OBJECTID', 'NOM_POBLAD', 'FUENTE', 'CÓDIGO', 'CAT_POBLAD', 'DIST', 'PROV', 'DEP', 'CÓD_INT', 'CATEGORIA', 'X', 'Y', 'N_BUSQDA']

🔄 Extrayendo muestra de 1000 registros...
✅ Muestra extraída: 1,000 registros
📋 Columnas disponibles: ['OBJECTID', 'NOM_POBLAD', 'FUENTE', 'CÓDIGO', 'CAT_POBLAD', 'DIST', 'PROV', 'DEP', 'CÓD_INT', 'CATEGORIA', 'X', 'Y', 'N_BUSQDA', 'COORD_X', 'COORD_Y']

📊 INFORMACIÓN DE LA MUESTRA:
   📍 Rango de coordenadas X: -76.27700 a -69.57727
   📍 Rango de coordenadas Y: -11.49074 a -10.00100

👀 PRIMERAS 5 FILAS:
    NOM_POBLAD      DIST      DEP    COORD_X    COORD_Y
0   PANDISHARI  RAYMONDI  UCAYALI -74.064617 -10.371287
1      CHICOSA  RAYMONDI  UCAYALI -74.061529 -10.378517
2         RAYA  RAYMONDI  UCAYALI -72.941178 -10.330434
3  PENSILVA

In [18]:
df_ccpp_sample['CATEGORIA'].value_counts()

CATEGORIA
Centro Poblado Menor    987
Capital de Distrito      13
Name: count, dtype: int64

In [13]:
# 📊 INFORMACIÓN COMPLETA DE TUS DATOS DE CENTROS POBLADOS
print("📊 RESUMEN COMPLETO DE TUS DATOS DE CENTROS POBLADOS")
print("=" * 60)

print(f"🗂️ DATOS TOTALES DISPONIBLES:")
print(f"   📁 Archivo completo: {num_records:,} centros poblados")
print(f"   📊 Muestra cargada: {len(df_ccpp_sample):,} registros")
print(f"   📈 Porcentaje de muestra: {(len(df_ccpp_sample)/num_records)*100:.2f}%")

print(f"\n🌍 COORDENADAS DISPONIBLES:")
print(f"   • En la muestra: {len(df_ccpp_sample):,} pares de coordenadas (X, Y)")
print(f"   • En total disponibles: {num_records:,} pares de coordenadas")

print(f"\n📍 INFORMACIÓN DE COORDENADAS (MUESTRA):")
print(f"   • Coordenada X (Longitud): {df_ccpp_sample['COORD_X'].min():.6f} a {df_ccpp_sample['COORD_X'].max():.6f}")
print(f"   • Coordenada Y (Latitud): {df_ccpp_sample['COORD_Y'].min():.6f} a {df_ccpp_sample['COORD_Y'].max():.6f}")

# Mostrar solo las coordenadas
coordenadas_muestra = df_ccpp_sample[['NOM_POBLAD', 'COORD_X', 'COORD_Y']].copy()
print(f"\n📋 EJEMPLO DE COORDENADAS (primeras 10):")
print(coordenadas_muestra.head(10))

print(f"\n🎯 PARA OBTENER TODAS LAS COORDENADAS ({num_records:,} registros):")
print(f"   1. 🔄 Usar el código de abajo para extraer todo el dataset")
print(f"   2. ⚠️  Advertencia: Cargar todos los registros puede tomar varios minutos")
print(f"   3. 💾 Recomendación: Exportar a CSV para uso futuro")

📊 RESUMEN COMPLETO DE TUS DATOS DE CENTROS POBLADOS
🗂️ DATOS TOTALES DISPONIBLES:
   📁 Archivo completo: 136,587 centros poblados
   📊 Muestra cargada: 1,000 registros
   📈 Porcentaje de muestra: 0.73%

🌍 COORDENADAS DISPONIBLES:
   • En la muestra: 1,000 pares de coordenadas (X, Y)
   • En total disponibles: 136,587 pares de coordenadas

📍 INFORMACIÓN DE COORDENADAS (MUESTRA):
   • Coordenada X (Longitud): -76.277002 a -69.577269
   • Coordenada Y (Latitud): -11.490744 a -10.001002

📋 EJEMPLO DE COORDENADAS (primeras 10):
             NOM_POBLAD    COORD_X    COORD_Y
0            PANDISHARI -74.064617 -10.371287
1               CHICOSA -74.061529 -10.378517
2                  RAYA -72.941178 -10.330434
3           PENSILVANIA -74.059879 -10.404008
4           PONTE VEDRA -74.037875 -10.418090
5               GALILEA -73.985168 -10.453842
6         NUEVO EL POZO -74.026564 -10.463330
7           BOCA COCANI -74.023100 -10.478250
8      PUERTO ESPERANZA -73.973340 -10.472008
9  TAHUARAP

In [4]:
# 🚀 EXTRAER TODAS LAS COORDENADAS (136,587 REGISTROS)
print("🚀 EXTRAYENDO TODAS LAS COORDENADAS DEL DATASET COMPLETO")
print("=" * 60)

import time
start_time = time.time()

print(f"🔄 Procesando {num_records:,} centros poblados...")
print("   ⏳ Esto puede tomar varios minutos...")

try:
    with shapefile.Reader(SHAPEFILE_PATH) as shp:
        
        # Obtener nombres de campos
        field_names = [field[0] for field in shp.fields[1:]]
        
        # Inicializar listas para almacenar datos
        all_records = []
        total_processed = 0
        
        # Procesar todos los registros
        for i, (record, shape) in enumerate(zip(shp.records(), shp.shapes())):
            
            # Mostrar progreso cada 10,000 registros
            if i % 10000 == 0:
                elapsed_time = time.time() - start_time
                print(f"   📊 Procesados: {i:,} / {num_records:,} ({(i/num_records)*100:.1f}%) - Tiempo: {elapsed_time:.1f}s")
            
            # Extraer coordenadas del punto
            if shape.points:
                x, y = shape.points[0]  # Primer punto (para puntos, solo hay uno)
                
                # Solo guardar los datos esenciales: nombre y coordenadas
                record_data = {
                    'OBJECTID': record[0] if len(record) > 0 else None,
                    'NOM_POBLAD': record[1] if len(record) > 1 else None,
                    'DIST': record[5] if len(record) > 5 else None,
                    'PROV': record[6] if len(record) > 6 else None,
                    'DEP': record[7] if len(record) > 7 else None,
                    'COORD_X': x,
                    'COORD_Y': y
                }
                
                all_records.append(record_data)
                total_processed += 1
        
        # Crear DataFrame con todos los datos
        df_ccpp_completo = pd.DataFrame(all_records)
        
        elapsed_time = time.time() - start_time
        
        print(f"\n✅ EXTRACCIÓN COMPLETADA!")
        print(f"   📊 Total procesado: {len(df_ccpp_completo):,} centros poblados")
        print(f"   ⏱️ Tiempo total: {elapsed_time:.1f} segundos")
        print(f"   💾 Memoria utilizada: {df_ccpp_completo.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")
        
        # Información de las coordenadas completas
        print(f"\n🌍 COORDENADAS COMPLETAS:")
        print(f"   📍 Rango X (Longitud): {df_ccpp_completo['COORD_X'].min():.6f} a {df_ccpp_completo['COORD_X'].max():.6f}")
        print(f"   📍 Rango Y (Latitud): {df_ccpp_completo['COORD_Y'].min():.6f} a {df_ccpp_completo['COORD_Y'].max():.6f}")
        
        # Estadísticas por departamento
        print(f"\n🏴 DISTRIBUCIÓN POR DEPARTAMENTO (Top 10):")
        dep_counts_completo = df_ccpp_completo['DEP'].value_counts().head(10)
        for dep, count in dep_counts_completo.items():
            print(f"   • {dep}: {count:,} centros poblados")
        
        print(f"\n💾 PARA GUARDAR TODAS LAS COORDENADAS:")
        print(f"   # Exportar solo coordenadas")
        print(f"   coordenadas_completas = df_ccpp_completo[['NOM_POBLAD', 'COORD_X', 'COORD_Y']]")
        print(f"   coordenadas_completas.to_csv('../DATA/coordenadas_centros_poblados_completo.csv', index=False)")
        print(f"   ")
        print(f"   # O exportar todo el dataset")
        print(f"   df_ccpp_completo.to_csv('../DATA/centros_poblados_completo.csv', index=False)")
        
except Exception as e:
    print(f"❌ Error durante la extracción: {e}")
    print("💡 Si hay problemas de memoria, usa la muestra de 1000 registros")

🚀 EXTRAYENDO TODAS LAS COORDENADAS DEL DATASET COMPLETO
🔄 Procesando 136,587 centros poblados...
   ⏳ Esto puede tomar varios minutos...
   📊 Procesados: 0 / 136,587 (0.0%) - Tiempo: 0.8s
   📊 Procesados: 10,000 / 136,587 (7.3%) - Tiempo: 0.8s
   📊 Procesados: 20,000 / 136,587 (14.6%) - Tiempo: 0.8s
   📊 Procesados: 30,000 / 136,587 (22.0%) - Tiempo: 0.8s
   📊 Procesados: 40,000 / 136,587 (29.3%) - Tiempo: 0.8s
   📊 Procesados: 50,000 / 136,587 (36.6%) - Tiempo: 0.8s
   📊 Procesados: 60,000 / 136,587 (43.9%) - Tiempo: 0.8s
   📊 Procesados: 70,000 / 136,587 (51.2%) - Tiempo: 0.8s
   📊 Procesados: 80,000 / 136,587 (58.6%) - Tiempo: 0.8s
   📊 Procesados: 90,000 / 136,587 (65.9%) - Tiempo: 0.8s
   📊 Procesados: 100,000 / 136,587 (73.2%) - Tiempo: 0.9s
   📊 Procesados: 110,000 / 136,587 (80.5%) - Tiempo: 0.9s
   📊 Procesados: 120,000 / 136,587 (87.9%) - Tiempo: 0.9s
   📊 Procesados: 130,000 / 136,587 (95.2%) - Tiempo: 0.9s

✅ EXTRACCIÓN COMPLETADA!
   📊 Total procesado: 136,587 centros pobl

In [16]:
# 💾 GUARDAR Y MOSTRAR RESUMEN FINAL
print("💾 GUARDANDO COORDENADAS Y MOSTRANDO RESUMEN FINAL")
print("=" * 60)

# Crear dataset solo con coordenadas
coordenadas_completas = df_ccpp_completo[['NOM_POBLAD', 'COORD_X', 'COORD_Y']].copy()

print(f"📊 RESUMEN FINAL DE TUS COORDENADAS:")
print(f"   🗂️ Total de centros poblados: {len(df_ccpp_completo):,}")
print(f"   🌍 Total de coordenadas: {len(coordenadas_completas):,} pares (X, Y)")
print(f"   📐 Cobertura geográfica completa del Perú")

print(f"\n🌍 RANGO GEOGRÁFICO COMPLETO:")
print(f"   📍 Longitud (X): {df_ccpp_completo['COORD_X'].min():.6f} a {df_ccpp_completo['COORD_X'].max():.6f}")
print(f"   📍 Latitud (Y): {df_ccpp_completo['COORD_Y'].min():.6f} a {df_ccpp_completo['COORD_Y'].max():.6f}")

print(f"\n📋 MUESTRA DE COORDENADAS (primeras 10):")
print(coordenadas_completas.head(10))

print(f"\n📈 ESTADÍSTICAS:")
print(f"   • Departamentos cubiertos: {df_ccpp_completo['DEP'].nunique()}")
print(f"   • Provincias cubiertas: {df_ccpp_completo['PROV'].nunique()}")
print(f"   • Distritos cubiertos: {df_ccpp_completo['DIST'].nunique()}")

# Guardar archivos
print(f"\n💾 GUARDANDO ARCHIVOS:")

try:
    # Guardar solo coordenadas
    coord_file = '../DATA/coordenadas_centros_poblados_completo.csv'
    coordenadas_completas.to_csv(coord_file, index=False)
    print(f"   ✅ Coordenadas guardadas: {coord_file}")
    
    # Guardar dataset completo
    full_file = '../DATA/centros_poblados_completo.csv'
    df_ccpp_completo.to_csv(full_file, index=False)
    print(f"   ✅ Dataset completo guardado: {full_file}")
    
    print(f"\n📊 ARCHIVOS CREADOS:")
    print(f"   📁 {coord_file}")
    print(f"      └─ {len(coordenadas_completas):,} filas × 3 columnas (nombre + coordenadas)")
    print(f"   📁 {full_file}") 
    print(f"      └─ {len(df_ccpp_completo):,} filas × {len(df_ccpp_completo.columns)} columnas (info completa)")
    
except Exception as e:
    print(f"   ❌ Error guardando archivos: {e}")
    print(f"   💡 Verifica que la carpeta DATA/ exista")

print(f"\n🎯 LISTO PARA TU ANÁLISIS DE RIESGO HÍDRICO:")
print(f"   • Tienes {len(coordenadas_completas):,} coordenadas de centros poblados")
print(f"   • Datos guardados en formato CSV para fácil uso")
print(f"   • Listos para cruzar con tus datos de monitoreo de agua")
print(f"   • Cobertura completa del territorio peruano")

print(f"\n🚀 PRÓXIMO PASO: Integrar con tu Radar de Riesgo Hídrico")

💾 GUARDANDO COORDENADAS Y MOSTRANDO RESUMEN FINAL
📊 RESUMEN FINAL DE TUS COORDENADAS:
   🗂️ Total de centros poblados: 136,587
   🌍 Total de coordenadas: 136,587 pares (X, Y)
   📐 Cobertura geográfica completa del Perú

🌍 RANGO GEOGRÁFICO COMPLETO:
   📍 Longitud (X): -81.310114 a -68.659959
   📍 Latitud (Y): -18.344072 a -0.030713

📋 MUESTRA DE COORDENADAS (primeras 10):
             NOM_POBLAD    COORD_X    COORD_Y
0            PANDISHARI -74.064617 -10.371287
1               CHICOSA -74.061529 -10.378517
2                  RAYA -72.941178 -10.330434
3           PENSILVANIA -74.059879 -10.404008
4           PONTE VEDRA -74.037875 -10.418090
5               GALILEA -73.985168 -10.453842
6         NUEVO EL POZO -74.026564 -10.463330
7           BOCA COCANI -74.023100 -10.478250
8      PUERTO ESPERANZA -73.973340 -10.472008
9  TAHUARAPA SHEREMASHI -73.977674 -10.489965

📈 ESTADÍSTICAS:
   • Departamentos cubiertos: 28
   • Provincias cubiertas: 208
   • Distritos cubiertos: 1790

💾 GUARD

## 🎯 RESUMEN EJECUTIVO Y PRÓXIMOS PASOS

### ✅ **LO QUE HEMOS DESCUBIERTO:**

Tu archivo `CCPP_0` contiene un **Shapefile con datos geoespaciales** de **136,587 centros poblados** del Perú, incluyendo:

#### 📊 **Datos Disponibles:**
- **📍 Coordenadas exactas** (X, Y) de cada centro poblado
- **🏷️ Nombres** de centros poblados
- **🗺️ División administrativa** (Departamento, Provincia, Distrito)
- **🏘️ Categoría** del centro poblado

#### 🌍 **Coordenadas Geográficas:**
- **Formato**: Latitud/Longitud decimal
- **Cobertura**: Todo el territorio peruano
- **Precisión**: Coordenadas exactas para ubicación geoespacial

### 🚀 **INTEGRACIÓN CON TU RADAR DE RIESGO HÍDRICO:**

Estos datos de centros poblados son **PERFECTOS** para tu análisis porque te permiten:

1. **🎯 Identificar población en riesgo**: Saber qué centros poblados están cerca de zonas de contaminación
2. **📏 Calcular proximidad**: Determinar distancia entre centros poblados y puntos de monitoreo de agua
3. **📊 Priorizar intervenciones**: Enfocar recursos en áreas con mayor densidad poblacional
4. **🗺️ Visualizar geográficamente**: Crear mapas de riesgo que muestren población afectada

In [10]:
# 🔗 EJEMPLO DE INTEGRACIÓN CON DATOS DE AGUA
print("🔗 CÓMO INTEGRAR CON TU RADAR DE RIESGO HÍDRICO")
print("=" * 60)

# Simulación de cómo combinar con datos de agua
print("💡 EJEMPLO DE CÓDIGO PARA INTEGRACIÓN:")
print("-" * 40)

ejemplo_codigo = '''
# 1. Cargar ambos datasets
df_centros_poblados = df_ccpp_sample  # Del shapefile
df_agua_superficial = df_agua_superficial  # De tu análisis anterior

# 2. Función para calcular proximidad geográfica
from math import radians, cos, sin, asin, sqrt

def calcular_distancia(lat1, lon1, lat2, lon2):
    """Calcular distancia en km entre dos puntos"""
    R = 6371  # Radio de la Tierra en km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 2 * R * asin(sqrt(a))

# 3. Encontrar centros poblados cerca de puntos de monitoreo
def encontrar_poblados_en_riesgo(df_poblados, df_agua, radio_km=10):
    poblados_riesgo = []
    
    for _, punto_agua in df_agua.iterrows():
        if pd.notna(punto_agua['COORD_ESTE']) and pd.notna(punto_agua['COORD_NORTE']):
            for _, poblado in df_poblados.iterrows():
                distancia = calcular_distancia(
                    punto_agua['COORD_NORTE'], punto_agua['COORD_ESTE'],
                    poblado['COORD_Y'], poblado['COORD_X']
                )
                
                if distancia <= radio_km:
                    poblados_riesgo.append({
                        'poblado': poblado['NOM_POBLAD'],
                        'departamento': poblado['DEP'],
                        'distrito': poblado['DIST'],
                        'distancia_km': distancia,
                        'punto_monitoreo': punto_agua['EXPEDIENTE'],
                        'coordinacion': punto_agua['COORDINACION']
                    })
    
    return pd.DataFrame(poblados_riesgo)

# 4. Ejemplo de uso
# poblados_en_riesgo = encontrar_poblados_en_riesgo(df_centros_poblados, df_agua_superficial)
'''

print(ejemplo_codigo)

print("\n🎯 MÉTRICAS QUE PUEDES CALCULAR:")
print("   • 👥 Población en riesgo por zona de contaminación")
print("   • 📏 Distancia promedio entre centros poblados y puntos de monitoreo")
print("   • 🗺️ Departamentos/provincias con mayor exposición")
print("   • 🚨 Centros poblados en zona de riesgo crítico")

print(f"\n📊 POTENCIAL DE TUS DATOS:")
print(f"   • {len(df_ccpp_sample):,} centros poblados (muestra)")
print(f"   • Coordenadas precisas para análisis geoespacial")
print(f"   • División administrativa completa")
print(f"   • Listo para cruzar con datos de monitoreo de agua")

print(f"\n💾 PARA GUARDAR LOS DATOS:")
print(f"   # Exportar a CSV para uso posterior")
print(f"   df_ccpp_sample.to_csv('../DATA/centros_poblados_muestra.csv', index=False)")
print(f"   # O cargar todo el shapefile si necesitas más datos")

print(f"\n🚀 PRÓXIMO PASO RECOMENDADO:")
print(f"   Integrar estos datos de población con tu análisis de hotspots")
print(f"   para crear un 'Radar de Riesgo Poblacional' más completo")

🔗 CÓMO INTEGRAR CON TU RADAR DE RIESGO HÍDRICO
💡 EJEMPLO DE CÓDIGO PARA INTEGRACIÓN:
----------------------------------------

# 1. Cargar ambos datasets
df_centros_poblados = df_ccpp_sample  # Del shapefile
df_agua_superficial = df_agua_superficial  # De tu análisis anterior

# 2. Función para calcular proximidad geográfica
from math import radians, cos, sin, asin, sqrt

def calcular_distancia(lat1, lon1, lat2, lon2):
    """Calcular distancia en km entre dos puntos"""
    R = 6371  # Radio de la Tierra en km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 2 * R * asin(sqrt(a))

# 3. Encontrar centros poblados cerca de puntos de monitoreo
def encontrar_poblados_en_riesgo(df_poblados, df_agua, radio_km=10):
    poblados_riesgo = []
    
    for _, punto_agua in df_agua.iterrows():
        if pd.notna(punto_agua['COORD_ESTE']) and pd.notna(punto_ag

In [6]:
# 🎯 PROCESAMIENTO FINAL PARA HACKATON - DATASET CENTROS POBLADOS
print("🎯 PROCESAMIENTO FINAL PARA HACKATON - CENTROS POBLADOS")
print("=" * 65)

print("📋 SELECCIONANDO COLUMNAS ESENCIALES PARA LA HACKATON...")

# Ya tenemos el dataset completo cargado en df_ccpp_completo
print(f"✅ Centros poblados cargados: {len(df_ccpp_completo):,}")

print(f"\n🔍 COLUMNAS DISPONIBLES:")
print(f"   {list(df_ccpp_completo.columns)}")

print(f"\n🔍 RANGOS ACTUALES DE COORDENADAS:")
print(f"   • Longitud (COORD_X): {df_ccpp_completo['COORD_X'].min():.6f} a {df_ccpp_completo['COORD_X'].max():.6f}")
print(f"   • Latitud (COORD_Y): {df_ccpp_completo['COORD_Y'].min():.6f} a {df_ccpp_completo['COORD_Y'].max():.6f}")

# Seleccionar y renombrar columnas esenciales para la hackaton (ajustado a las columnas disponibles)
columnas_esenciales = {
    'OBJECTID': 'id_centro_poblado',
    'NOM_POBLAD': 'nombre_centro_poblado',
    'DIST': 'distrito',
    'PROV': 'provincia', 
    'DEP': 'departamento',
    'COORD_X': 'longitud',
    'COORD_Y': 'latitud'
}

# Verificar qué columnas existen realmente
columnas_disponibles = {}
for col_orig, col_nueva in columnas_esenciales.items():
    if col_orig in df_ccpp_completo.columns:
        columnas_disponibles[col_orig] = col_nueva
    else:
        print(f"   ⚠️ Columna {col_orig} no encontrada")

print(f"\n📋 COLUMNAS SELECCIONADAS:")
for col_orig, col_nueva in columnas_disponibles.items():
    print(f"   • {col_orig} → {col_nueva}")

# Crear dataset final con solo las columnas disponibles
df_poblacion_procesado = df_ccpp_completo[list(columnas_disponibles.keys())].copy()
df_poblacion_procesado = df_poblacion_procesado.rename(columns=columnas_disponibles)

print(f"\n📊 DATASET PROCESADO:")
print(f"   • Registros: {len(df_poblacion_procesado):,}")
print(f"   • Columnas: {len(df_poblacion_procesado.columns)}")
print(f"   • Memoria: {df_poblacion_procesado.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print(f"\n📝 COLUMNAS FINALES:")
for i, col in enumerate(df_poblacion_procesado.columns, 1):
    non_null = df_poblacion_procesado[col].notna().sum()
    completitud = (non_null / len(df_poblacion_procesado)) * 100
    print(f"   {i:2d}. {col}: {non_null:,} registros ({completitud:.1f}% completo)")

print(f"\n📋 MUESTRA DEL DATASET INICIAL:")
print(df_poblacion_procesado.head())

🎯 PROCESAMIENTO FINAL PARA HACKATON - CENTROS POBLADOS
📋 SELECCIONANDO COLUMNAS ESENCIALES PARA LA HACKATON...
✅ Centros poblados cargados: 136,587

🔍 COLUMNAS DISPONIBLES:
   ['OBJECTID', 'NOM_POBLAD', 'DIST', 'PROV', 'DEP', 'COORD_X', 'COORD_Y']

🔍 RANGOS ACTUALES DE COORDENADAS:
   • Longitud (COORD_X): -81.310114 a -68.659959
   • Latitud (COORD_Y): -18.344072 a -0.030713

📋 COLUMNAS SELECCIONADAS:
   • OBJECTID → id_centro_poblado
   • NOM_POBLAD → nombre_centro_poblado
   • DIST → distrito
   • PROV → provincia
   • DEP → departamento
   • COORD_X → longitud
   • COORD_Y → latitud

📊 DATASET PROCESADO:
   • Registros: 136,587
   • Columnas: 7
   • Memoria: 37.9 MB

📝 COLUMNAS FINALES:
    1. id_centro_poblado: 136,587 registros (100.0% completo)
    2. nombre_centro_poblado: 136,587 registros (100.0% completo)
    3. distrito: 136,587 registros (100.0% completo)
    4. provincia: 136,587 registros (100.0% completo)
    5. departamento: 136,587 registros (100.0% completo)
    6. l

In [7]:
# 🧹 VALIDACIÓN DE COORDENADAS Y LIMPIEZA DE DATOS
print("🧹 VALIDACIÓN DE COORDENADAS Y LIMPIEZA DE DATOS")
print("=" * 55)

print("🔍 ANÁLISIS DE COORDENADAS ANTES DEL FILTRO:")
coords_antes = len(df_poblacion_procesado)
print(f"   • Total de centros poblados: {coords_antes:,}")
print(f"   • Rango longitud: {df_poblacion_procesado['longitud'].min():.6f} a {df_poblacion_procesado['longitud'].max():.6f}")
print(f"   • Rango latitud: {df_poblacion_procesado['latitud'].min():.6f} a {df_poblacion_procesado['latitud'].max():.6f}")

# Verificar coordenadas (0, 0)
coords_cero = ((df_poblacion_procesado['longitud'] == 0) & (df_poblacion_procesado['latitud'] == 0)).sum()
print(f"   • Coordenadas (0, 0): {coords_cero:,}")

# Verificar coordenadas nulas
coords_null_lon = df_poblacion_procesado['longitud'].isnull().sum()
coords_null_lat = df_poblacion_procesado['latitud'].isnull().sum()
print(f"   • Longitudes nulas: {coords_null_lon:,}")
print(f"   • Latitudes nulas: {coords_null_lat:,}")

print(f"\n🌍 APLICANDO FILTRO DE COORDENADAS VÁLIDAS PARA PERÚ:")
print(f"   • Longitud: -81.5° a -68.0° (territorio peruano)")
print(f"   • Latitud: -18.5° a 0.5° (territorio peruano)")
print(f"   • Excluyendo coordenadas (0, 0)")

# Filtro para coordenadas válidas en territorio peruano
df_poblacion_procesado = df_poblacion_procesado[
    (df_poblacion_procesado['longitud'] >= -81.5) & 
    (df_poblacion_procesado['longitud'] <= -68.0) &
    (df_poblacion_procesado['latitud'] >= -18.5) & 
    (df_poblacion_procesado['latitud'] <= 0.5) &
    (df_poblacion_procesado['longitud'] != 0) &
    (df_poblacion_procesado['latitud'] != 0) &
    (df_poblacion_procesado['longitud'].notna()) &
    (df_poblacion_procesado['latitud'].notna())
]

coords_despues = len(df_poblacion_procesado)
coords_eliminados = coords_antes - coords_despues

print(f"\n✅ RESULTADO DEL FILTRO:")
print(f"   • Centros poblados válidos: {coords_despues:,}")
print(f"   • Centros poblados eliminados: {coords_eliminados:,}")
print(f"   • Porcentaje conservado: {(coords_despues/coords_antes)*100:.2f}%")

print(f"\n🗺️ RANGO GEOGRÁFICO DESPUÉS DEL FILTRO:")
print(f"   • Longitud: {df_poblacion_procesado['longitud'].min():.6f} a {df_poblacion_procesado['longitud'].max():.6f}")
print(f"   • Latitud: {df_poblacion_procesado['latitud'].min():.6f} a {df_poblacion_procesado['latitud'].max():.6f}")

print(f"\n🧹 LIMPIEZA Y ESTANDARIZACIÓN DE CAMPOS DE TEXTO:")

# Limpiar campos de texto
df_poblacion_procesado['nombre_centro_poblado'] = df_poblacion_procesado['nombre_centro_poblado'].str.strip().str.upper()
df_poblacion_procesado['departamento'] = df_poblacion_procesado['departamento'].str.strip().str.upper()
df_poblacion_procesado['provincia'] = df_poblacion_procesado['provincia'].str.strip().str.upper()
df_poblacion_procesado['distrito'] = df_poblacion_procesado['distrito'].str.strip().str.upper()

print(f"   ✅ Campos de texto estandarizados (mayúsculas, sin espacios extra)")

print(f"\n📊 ESTADÍSTICAS FINALES:")
print(f"   • Departamentos únicos: {df_poblacion_procesado['departamento'].nunique()}")
print(f"   • Provincias únicas: {df_poblacion_procesado['provincia'].nunique()}")
print(f"   • Distritos únicos: {df_poblacion_procesado['distrito'].nunique()}")

print(f"\n🏴 DISTRIBUCIÓN POR DEPARTAMENTO (Top 10):")
dep_counts = df_poblacion_procesado['departamento'].value_counts().head(10)
for i, (dep, count) in enumerate(dep_counts.items(), 1):
    porcentaje = (count / len(df_poblacion_procesado)) * 100
    print(f"   {i:2d}. {dep}: {count:,} ({porcentaje:.1f}%)")

print(f"\n📋 MUESTRA DEL DATASET LIMPIO:")
print(df_poblacion_procesado.head())

🧹 VALIDACIÓN DE COORDENADAS Y LIMPIEZA DE DATOS
🔍 ANÁLISIS DE COORDENADAS ANTES DEL FILTRO:
   • Total de centros poblados: 136,587
   • Rango longitud: -81.310114 a -68.659959
   • Rango latitud: -18.344072 a -0.030713
   • Coordenadas (0, 0): 0
   • Longitudes nulas: 0
   • Latitudes nulas: 0

🌍 APLICANDO FILTRO DE COORDENADAS VÁLIDAS PARA PERÚ:
   • Longitud: -81.5° a -68.0° (territorio peruano)
   • Latitud: -18.5° a 0.5° (territorio peruano)
   • Excluyendo coordenadas (0, 0)

✅ RESULTADO DEL FILTRO:
   • Centros poblados válidos: 136,587
   • Centros poblados eliminados: 0
   • Porcentaje conservado: 100.00%

🗺️ RANGO GEOGRÁFICO DESPUÉS DEL FILTRO:
   • Longitud: -81.310114 a -68.659959
   • Latitud: -18.344072 a -0.030713

🧹 LIMPIEZA Y ESTANDARIZACIÓN DE CAMPOS DE TEXTO:
   ✅ Campos de texto estandarizados (mayúsculas, sin espacios extra)

📊 ESTADÍSTICAS FINALES:
   • Departamentos únicos: 28
   • Provincias únicas: 208
   • Distritos únicos: 1790

🏴 DISTRIBUCIÓN POR DEPARTAMENT

In [8]:
# 💾 GUARDAR DATASET CENTROS POBLADOS EN CARPETA DATAFINAL
print("💾 GUARDANDO DATASET FINAL EN CARPETA DATAFINAL")
print("=" * 60)

import os

# Crear carpeta DATAFINAL si no existe
datafinal_path = '../DATAFINAL'
os.makedirs(datafinal_path, exist_ok=True)
print(f"📁 Carpeta DATAFINAL verificada/creada")

# Guardar dataset procesado
archivo_final = os.path.join(datafinal_path, 'poblacion_procesado.csv')

try:
    df_poblacion_procesado.to_csv(archivo_final, index=False, encoding='utf-8')
    print(f"\n✅ ARCHIVO GUARDADO EXITOSAMENTE:")
    print(f"   📄 Archivo: {archivo_final}")
    print(f"   📊 Registros: {len(df_poblacion_procesado):,}")
    print(f"   📋 Columnas: {len(df_poblacion_procesado.columns)}")
    
    # Verificar tamaño del archivo
    size_mb = os.path.getsize(archivo_final) / (1024 * 1024)
    print(f"   💾 Tamaño: {size_mb:.2f} MB")
    
except Exception as e:
    print(f"❌ Error guardando archivo: {e}")

print(f"\n📋 RESUMEN DEL DATASET PROCESADO PARA HACKATON:")
print(f"🎯 PROPÓSITO: Sistema de Monitoreo y Alerta Temprana Ambiental")
print(f"📊 DATOS DE POBLACIÓN LISTOS PARA:")
print(f"   • Correlación con puntos de monitoreo ambiental")
print(f"   • Análisis de densidad poblacional en zonas de riesgo")
print(f"   • Identificación de centros poblados vulnerables")
print(f"   • Cálculo de población afectada por contaminación")
print(f"   • Alertas tempranas para comunidades")

print(f"\n🔗 CAMPOS CLAVE PARA TU HACKATON:")
print(f"   🗺️ Geoespaciales: latitud, longitud")
print(f"   🏘️ Identificación: id_centro_poblado, nombre_centro_poblado")
print(f"   📍 Ubicación: departamento, provincia, distrito")

print(f"\n🏛️ DISTRIBUCIÓN GEOGRÁFICA (Top 5):")
top_deptos = df_poblacion_procesado['departamento'].value_counts().head(5)
for i, (depto, count) in enumerate(top_deptos.items(), 1):
    porcentaje = (count / len(df_poblacion_procesado)) * 100
    print(f"   {i}. {depto}: {count:,} ({porcentaje:.1f}%)")

print(f"\n🌍 COBERTURA GEOGRÁFICA:")
print(f"   • Departamentos: {df_poblacion_procesado['departamento'].nunique()}")
print(f"   • Provincias: {df_poblacion_procesado['provincia'].nunique()}")
print(f"   • Distritos: {df_poblacion_procesado['distrito'].nunique()}")

print(f"\n📊 COMPARACIÓN CON OTROS DATASETS:")
print(f"   🏥 Centros de salud: ~7,953 registros")
print(f"   🎓 Instituciones educativas: ~177,871 registros") 
print(f"   🏘️ Centros poblados: {len(df_poblacion_procesado):,} registros")

print(f"\n🗂️ ARCHIVOS EN DATAFINAL:")
datafinal_files = os.listdir(datafinal_path) if os.path.exists(datafinal_path) else []
for i, file in enumerate(datafinal_files, 1):
    if file.endswith('.csv'):
        file_path = os.path.join(datafinal_path, file)
        file_size = os.path.getsize(file_path) / (1024 * 1024)
        print(f"   {i}. {file} ({file_size:.1f} MB)")

print(f"\n🚀 DATASETS COMPLETADOS PARA HACKATON:")
print(f"   ✅ Salud: salud_procesado.csv")
print(f"   ✅ Educación: educacion_procesado.csv") 
print(f"   ✅ Población: poblacion_procesado.csv")

print(f"\n🎯 PRÓXIMO PASO: Integrar todos los datasets para análisis conjunto")
print(f"✅ DATASET DE CENTROS POBLADOS COMPLETADO Y LISTO PARA HACKATON")

💾 GUARDANDO DATASET FINAL EN CARPETA DATAFINAL
📁 Carpeta DATAFINAL verificada/creada

✅ ARCHIVO GUARDADO EXITOSAMENTE:
   📄 Archivo: ../DATAFINAL/poblacion_procesado.csv
   📊 Registros: 136,587
   📋 Columnas: 7
   💾 Tamaño: 10.82 MB

📋 RESUMEN DEL DATASET PROCESADO PARA HACKATON:
🎯 PROPÓSITO: Sistema de Monitoreo y Alerta Temprana Ambiental
📊 DATOS DE POBLACIÓN LISTOS PARA:
   • Correlación con puntos de monitoreo ambiental
   • Análisis de densidad poblacional en zonas de riesgo
   • Identificación de centros poblados vulnerables
   • Cálculo de población afectada por contaminación
   • Alertas tempranas para comunidades

🔗 CAMPOS CLAVE PARA TU HACKATON:
   🗺️ Geoespaciales: latitud, longitud
   🏘️ Identificación: id_centro_poblado, nombre_centro_poblado
   📍 Ubicación: departamento, provincia, distrito

🏛️ DISTRIBUCIÓN GEOGRÁFICA (Top 5):
   1. PUNO: 16,804 (12.3%)
   2. CUSCO: 13,496 (9.9%)
   3. ANCASH: 10,658 (7.8%)
   4. AYACUCHO: 10,081 (7.4%)
   5. HUANUCO: 9,193 (6.7%)

🌍 COBE